## MISA (2024-2025)
- Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

- Izay misy hoe `YOUR CODE HERE` na `YOUR ANSWER HERE` ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
 * [Introduction to Logistic Regression](https://en.wikipedia.org/wiki/Multinomial_logistic_regression)
 * [Introduction to Logistic Regression](https://courses.lumenlearning.com/introstats1/chapter/introduction-to-logistic-regression/#:~:text=Logistic%20regression%20is%20a%20type,different%20from%20the%20normal%20distribution)
 * [Loss Functions in Machine Learning](https://www.geeksforgeeks.org/loss-functions-in-deep-learning)
 * [Gradient Descent](https://en.wikipedia.org/wiki/Gradient_descent)
 * [Softmax regression](https://en.wikipedia.org/wiki/Softmax_function)


---

# Multinomial regression

In [2]:
from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.datasets import load_boston, load_diabetes, load_iris, load_digits
from scipy.special import huber
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def grad_check_sparse(f, x, analytic_grad, num_checks=12, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [3]:
data = load_iris()
X_train2, y_train2 = data.data, data.target

W = np.random.randn(X_train2.shape[1], 3) * 0.0001

## Naive

In [4]:
def softmax_loss_naive(W, X, y, alpha):
    """
    Softmax loss function WITH FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W; same shape as W
    """
    # Initialization
    num_classes = W.shape[1]
    num_train = X.shape[0]
    loss = 0.0
    dW = np.zeros_like(W)

    for i in range(num_train):
        # Compute the scores for each class
        scores = X[i].dot(W)

        # Prevent numerical instability by subtracting max score
        scores -= np.max(scores)

        # Compute the softmax probabilities
        exp_scores = np.exp(scores)
        probs = exp_scores / np.sum(exp_scores)

        # Compute the loss for the correct class
        loss -= np.log(probs[y[i]])

        # Compute the gradient for each class
        for j in range(num_classes):
            if j == y[i]:
                dW[:, j] += (probs[j] - 1) * X[i]
            else:
                dW[:, j] += probs[j] * X[i]

    # Average the loss and gradients over the number of training examples
    loss /= num_train
    dW /= num_train

    # Add regularization to the loss and gradient
    loss += 0.5 * alpha * np.sum(W * W)
    dW += alpha * W

    return loss, dW


### Without regularization

In [5]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.167171 analytic: -0.167171, relative error: 2.947539e-10
numerical: -0.275190 analytic: -0.275190, relative error: 2.730880e-11
numerical: 0.764030 analytic: 0.764030, relative error: 6.211300e-11
numerical: -0.030729 analytic: -0.030729, relative error: 4.470522e-09
numerical: -0.124547 analytic: -0.124547, relative error: 2.391774e-10
numerical: 0.764030 analytic: 0.764030, relative error: 6.211300e-11
numerical: 0.095839 analytic: 0.095839, relative error: 1.123248e-10
numerical: 0.277176 analytic: 0.277176, relative error: 5.497121e-10
numerical: -0.246447 analytic: -0.246447, relative error: 5.812440e-10
numerical: -0.275190 analytic: -0.275190, relative error: 2.730880e-11
numerical: 0.764030 analytic: 0.764030, relative error: 6.211300e-11
numerical: 0.277176 analytic: 0.277176, relative error: 5.497121e-10


### With regularization

In [6]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: 0.028966 analytic: 0.028966, relative error: 1.353426e-09
numerical: -0.166984 analytic: -0.166984, relative error: 2.817379e-10
numerical: 0.028966 analytic: 0.028966, relative error: 1.353426e-09
numerical: -0.124735 analytic: -0.124735, relative error: 2.660126e-10
numerical: 0.096013 analytic: 0.096013, relative error: 1.687716e-10
numerical: -0.030942 analytic: -0.030942, relative error: 4.601240e-09
numerical: -0.246217 analytic: -0.246217, relative error: 5.864328e-10
numerical: 0.096013 analytic: 0.096013, relative error: 1.687716e-10
numerical: -0.274926 analytic: -0.274926, relative error: 3.338375e-11
numerical: 0.317158 analytic: 0.317158, relative error: 8.815231e-11
numerical: -0.596998 analytic: -0.596998, relative error: 1.000548e-10
numerical: -0.124735 analytic: -0.124735, relative error: 2.660126e-10


## Vectorized

In [7]:
def softmax_loss_vectorized(W, X, y, alpha, fit_intercept=False):
    """
    Softmax loss function WITHOUT FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    # Compute scores
    scores = X.dot(W)
    scores -= np.max(scores, axis=1, keepdims=True)  # For numerical stability

    # Compute softmax probabilities
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute the loss
    N = X.shape[0]
    correct_log_probs = -np.log(probs[np.arange(N), y])
    loss = np.sum(correct_log_probs) / N
    loss += 0.5 * alpha * np.sum(W * W)

    # Compute the gradient
    dscores = probs
    dscores[np.arange(N), y] -= 1
    dscores /= N
    dW = X.T.dot(dscores)
    dW += alpha * W

    return loss, dW


### Without regularization

In [8]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.275190 analytic: -0.275190, relative error: 4.748076e-11
numerical: -0.275190 analytic: -0.275190, relative error: 4.748076e-11
numerical: 0.317351 analytic: 0.317351, relative error: 2.460166e-12
numerical: -0.275190 analytic: -0.275190, relative error: 4.748076e-11
numerical: -0.030729 analytic: -0.030729, relative error: 4.109230e-09
numerical: -0.275190 analytic: -0.275190, relative error: 4.748076e-11
numerical: -0.124547 analytic: -0.124547, relative error: 1.946067e-10
numerical: 0.764030 analytic: 0.764030, relative error: 6.211285e-11
numerical: -0.246447 analytic: -0.246447, relative error: 5.136696e-10
numerical: 0.317351 analytic: 0.317351, relative error: 2.460166e-12
numerical: -0.246447 analytic: -0.246447, relative error: 5.136696e-10
numerical: 0.764030 analytic: 0.764030, relative error: 6.211285e-11


### With regularization

In [9]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.166984 analytic: -0.166984, relative error: 3.482261e-10
numerical: -0.041942 analytic: -0.041942, relative error: 2.849529e-11
numerical: 0.764016 analytic: 0.764016, relative error: 5.876659e-11
numerical: 0.277011 analytic: 0.277011, relative error: 4.880493e-10
numerical: 0.764016 analytic: 0.764016, relative error: 5.876659e-11
numerical: 0.028966 analytic: 0.028966, relative error: 7.785096e-10
numerical: -0.246217 analytic: -0.246217, relative error: 5.187952e-10
numerical: -0.124735 analytic: -0.124735, relative error: 2.215092e-10
numerical: 0.764016 analytic: 0.764016, relative error: 5.876659e-11
numerical: -0.124735 analytic: -0.124735, relative error: 2.215092e-10
numerical: -0.030942 analytic: -0.030942, relative error: 4.242436e-09
numerical: -0.274926 analytic: -0.274926, relative error: 5.357502e-11


## Gradient descent

In [10]:
class LinearModel():
    def __init__(self, fit_intercept=True):
        self.W = None
        self.fit_intercept = fit_intercept

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        if self.fit_intercept:
            # Add a bias term by appending a column of ones to X
            X = np.hstack([np.ones((X.shape[0], 1)), X])

        N, d = X.shape
        C = (np.max(y) + 1)  # Number of classes

        if self.W is None:  # Initialize weights
            self.W = 0.001 * np.random.randn(d, C)

        # Run stochastic gradient descent to optimize W
        loss_history = []
        for it in range(num_iters):
            # Sample batch_size elements in X_batch and y_batch
            indices = np.random.choice(N, batch_size, replace=True)
            X_batch = X[indices]
            y_batch = y[indices]

            # Evaluate loss and gradient
            loss, dW = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # Perform parameter update
            self.W -= learning_rate * dW

            if verbose and it % 100 == 0:
                print(f"iteration {it} / {num_iters}: loss {loss}")

        return loss_history

    def predict(self, X):
        if self.fit_intercept:
            # Add a bias term to X
            X = np.hstack([np.ones((X.shape[0], 1)), X])

        # Compute class scores and select the class with the highest score
        scores = X.dot(self.W)
        y_pred = np.argmax(scores, axis=1)

        return y_pred

    def loss(self, X_batch, y_batch, reg):
        raise NotImplementedError("Subclasses should implement this method!")

class MultinomialLogisticRegressor(LinearModel):
    """ Softmax regression """

    def loss(self, X_batch, y_batch, alpha):
        return softmax_loss_vectorized(self.W, X_batch, y_batch, alpha)

    def predict(self, X):
        """
        Inputs:
        - X: array of shape (N, D)

        Returns:
        - y_pred: 1-dimensional array of length N, each element is an integer giving the predicted class
        """
        return super().predict(X)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train2)

sk_model = LogisticRegression(fit_intercept=False)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=False)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.01

iteration 0 / 75000: loss 1.098905045952543
iteration 100 / 75000: loss 0.9993766506973472
iteration 200 / 75000: loss 0.9444479018014497
iteration 300 / 75000: loss 0.8919579960560728
iteration 400 / 75000: loss 0.8047147297387693
iteration 500 / 75000: loss 0.8146247194914082
iteration 600 / 75000: loss 0.7480013028772616
iteration 700 / 75000: loss 0.7357646455203711
iteration 800 / 75000: loss 0.6460431102789556
iteration 900 / 75000: loss 0.6279178713420663
iteration 1000 / 75000: loss 0.6366190434443029
iteration 1100 / 75000: loss 0.6629978276182614
iteration 1200 / 75000: loss 0.6747941807102598
iteration 1300 / 75000: loss 0.5670818197962113
iteration 1400 / 75000: loss 0.6136295943852457
iteration 1500 / 75000: loss 0.5896559817655131
iteration 1600 / 75000: loss 0.5744535458657958
iteration 1700 / 75000: loss 0.5753446767524498
iteration 1800 / 75000: loss 0.5591741887310802
iteration 1900 / 75000: loss 0.5792989108627926
iteration 2000 / 75000: loss 0.5063092605611257
itera

In [12]:
sk_model = LogisticRegression(fit_intercept=True)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=True)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.02

iteration 0 / 75000: loss 1.0985757836893404
iteration 100 / 75000: loss 1.0093602324852786
iteration 200 / 75000: loss 0.9338753981716212
iteration 300 / 75000: loss 0.84486353219474
iteration 400 / 75000: loss 0.8414073992547166
iteration 500 / 75000: loss 0.7782176326457502
iteration 600 / 75000: loss 0.7134887965879351
iteration 700 / 75000: loss 0.7424957883871117
iteration 800 / 75000: loss 0.6885052527809503
iteration 900 / 75000: loss 0.722829136861787
iteration 1000 / 75000: loss 0.6640191007131884
iteration 1100 / 75000: loss 0.598943813231269
iteration 1200 / 75000: loss 0.6336991038609436
iteration 1300 / 75000: loss 0.5860456481287339
iteration 1400 / 75000: loss 0.5676641408368388
iteration 1500 / 75000: loss 0.6173317941175052
iteration 1600 / 75000: loss 0.5548884254851565
iteration 1700 / 75000: loss 0.5672416686411131
iteration 1800 / 75000: loss 0.5163700005940827
iteration 1900 / 75000: loss 0.5772274573770106
iteration 2000 / 75000: loss 0.5295104976828622
iteratio